# 텍스트 분류 (트위터단어분류+불용어 50개 적용) 

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re

In [2]:
from ckonlpy.tag import Twitter

In [3]:
# twitter = Twitter() # 형태소 분석기 

In [ ]:
# twitter.add_dictionary('현대자동차', 'Noun') # user dictionary 등록

In [5]:
# from ckonlpy.tag import Twitter
# from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# TfidfVectorizer.fit_transform()

In [7]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [8]:
df = pd.read_csv('new_data_8529.csv', sep='\t', encoding='UTF-8')

In [9]:
userdic = pd.read_csv('usedic.csv', sep='\t', encoding='UTF-8')

In [10]:
userdic['User Dic.'][0:1]

0    가니쉬
Name: User Dic., dtype: object

In [11]:
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})

In [12]:
userdic = userdic.fillna('')

In [13]:
userdic[:10]

stops standard missprint
0    결과      가니쉬       가니시
1    공임       가스        개스
2    광주       가죽          
3    넥쏘       간격        간극
4    니로       간격         갭
5    레이    간격 단차       간단차
6    렌트    간격 단차       갭단차
7   리무진    간격 단차       단간차
8    모닝       간섭          
9   모하비      간헐적       간혈적

In [14]:
userdic.shape

(656, 3)

In [15]:
userdic = userdic.astype(str).copy()

In [16]:
type(userdic)

pandas.core.frame.DataFrame

In [17]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [18]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---

In [19]:
df.shape

(8529, 17)

# 분류현황

In [20]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [21]:
category_count = df['구분'].value_counts()
category_count

기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [22]:
category_count = df['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [23]:
category_count = df['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# null인 값 확인 및 제거

In [24]:
df_bak = df.copy()

In [25]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [26]:
df.shape

(8529, 17)

In [27]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [28]:
category_count = df['구분'].value_counts()
category_count

N          5517
기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [29]:
category_count = df['책임구'].value_counts()
category_count

N       5517
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [30]:
category_count = df['원인'].value_counts()
category_count

N        5517
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# 전처리

In [31]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df['target_pred'] = df['대상 선정'].astype(str).copy()

In [32]:
df['target_pred'].head()

0    X
1    X
2    X
3    X
4    X
Name: target_pred, dtype: object

In [33]:
#명사 추출 함수
def nounextract(text):
    text = Okt.nouns(text)
    return text

In [34]:
df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nounextract)

In [35]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [36]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [37]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

Wall time: 250 ms


In [38]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 1.49 s


In [39]:
df['clm_prtl_sbc'].head()

0         엔진    룸    다량    연료    누유    발생    점검    ...
1               리    도어    앤티    취핑    필림    미장    착  
2         커튼    후크    미장    착       조치    내용    커튼 ...
3         및    조치       웨    스트립    고무    주름    발생 ...
4      몰딩    내부    고무    미조    립    상태    몰딩    조립 ...
Name: clm_prtl_sbc, dtype: object

In [66]:
df['clm_prtl_sbc'].shape

(8529,)

In [67]:
df['clm_prtl_sbc'][1]

'     리    도어    앤티    취핑    필림    미장    착  '

In [73]:
df['clm_prtl_sbc'].find('간극')

AttributeError: 'Series' object has no attribute 'find'

In [72]:
df[df['clm_prtl_sbc'].str.contains(userdic.missprint[1])].head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
3017               W  2018     20180713         ZZ7         I11   작동불량_작동안됨   
7417               W  2017     20170316         ZZ7         Q57   소음/이음_작동시   
7421               W  2018     20180616         ZZ7         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
3017         KV11          CK     J5          CC        ACGAS   
7417         KV11          YP     A9          CC        ACGAS   
7421         KV11          YP     A9          CC        ACGAS   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
3017       에어콘    작동    불량       발생       에어콘    개스 ...     X  N   N  N  X---   
7417       에어컨    쉬익    소리    발생    점검    내용       에...     X  N   N  N  X---   
7421       주행    에어콘    동시    소음    발생       에어콘    ...     X  N   N  N  X---   

     target_pred  
3017           X  
7417           X  
7421           X

In [ ]:
# 붙어 있는 단어 오자 검색후 replace 수정
index1=0
index2=0
data = df['clm_prtl_sbc']
for index2 in range(0,656):
    for index1 in range(0,8529):
        if (userdic.missprint[index2] == data[index1].str.contains(userdic.missprint[index2])):
            vocab[index1] = vocab[index1].replace(userdic.misspring[index2],userdic.standard[index2], True)
            index2+=1
        else:
            index1+=1

In [ ]:
# 낱개 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,8529):
        if (userdic.missprint[index2] == vocab[index1]):
            vocab[index1] = vocab[index1].replace(userdic.missprint[index2],userdic.standard[index2], True)            
            index2+=1
        else:
            index1+=1

In [40]:
# n_rows = len(df.index)
# target = df['clm_prtl_sbc']
# index = list(df.index)
# i=0
# for i in range(n_rows):
#     choice = index[i]
#     userdic_stop = usedic['User Dic.'][0:choice]
#     userdic_synom = usedic['동의어'][0:choice]
#     if (target.str.contains(userdic_stop[choice]) == userdic_stop[choice]):
#         df.replace(to_replace = userdic_stop ,value = userdic_synom)
#     else : 
#         i = i + 1 

In [41]:
# df2 = df[df['clm_prtl_sbc'].str.contains(usedic.동의어[1])]
# df2['clm_prtl_sbc'].replace(to_replace='개스',value='가스')

In [42]:
# df['clm_prtl_sbc'] = df2['clm_prtl_sbc'].replace(to_replace='간극',value='간격')

In [43]:
# df1 = df.replace(to_replace="가니시",value="가니쉬")
# df1 = df.replace(to_replace="개스",value="가스")
# df1 = df.replace(to_replace="간극",value="간격")
# df1 = df.replace(to_replace="간단차",value="간격 단차")
# df1 = df.replace(to_replace="갭단차",value="간격 단차")
# df1 = df.replace(to_replace="단간차",value="간격 단차")
# df1 = df.replace(to_replace="간혈적",value="간헐적")
# df1 = df.replace(to_replace="감지센서",value="감지 센서")
# df1 = df.replace(to_replace="울트라센서",value="감지 센서")
# df1 = df.replace(to_replace="울트라 센서",value="감지 센서")
# df1 = df.replace(to_replace="개페",value="개폐")
# df1 = df.replace(to_replace="개패",value="개폐")
# df1 = df.replace(to_replace="문구",value="고장코드")
# df1 = df.replace(to_replace="맨트",value="고장코드")
# df1 = df.replace(to_replace="멘트",value="고장코드")

# 학습데이터와 테스트데이터 만들기

In [44]:
df = df.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [45]:
df.shape

(8529, 18)

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
df_train_v,df_test_v = train_test_split(df, test_size = 0.1, random_state=123)

In [48]:
df_train = df_train_v.copy()
df_test = df_test_v.copy()

In [49]:
df_train.shape

(7676, 18)

In [50]:
df_test.shape

(853, 18)

# 단어 벡터화

In [51]:
stop_dict = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stop_dict, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['현상', '[C]', '현상및점검', '현    상', '현 상', '점검점검', '점검내용', '조치내용', '점검점검내용', '현상및조치', 'C', '원인', '점검/원인', '사진첨부', '(사진첨부)', '사진참조', '점검및원인', '동영상참조', '동영상', '대리점', '고객', '고객에게', '상기', '차량', '상기차량은', '자동차', '당차량은', '당차량', '당', '차량은', '점검및조치내용', 'F/LH', 'F/RH', 'L/H', 'R/H', 'L/H리어', 'R/H리어', ...'현  상', '현 상', '현산', '화성', '그랜저', '새벽', '새차', '야간', '주간', '그랜져', '주재원', '아침', '밤, 심야', '낮,낫', '주제원'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [53]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['100', '10000', 'h리어', 'lh', 'lh쪽', 'rh', 'rh쪽', '넘버', '리어lh', '리어rh', '마스터', '번호', '심야', '임시', '점검'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(7676, 20000)

In [54]:
#%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

(853, 20000)

In [55]:
type(train_feature_vector)

scipy.sparse.csr.csr_matrix

In [56]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['가감',
 '가감 실내',
 '가감 실내 대쉬보드',
 '가감 저속',
 '가감 저속 등판',
 '가끔',
 '가끔 소음',
 '가끔 소음 허브',
 '가끔 재현',
 '가끔 재현 발생']

In [57]:
len(vocab)

20000

In [58]:
userdic.missprint[0]

'가니시'

In [59]:
userdic.standard[0]

'가니쉬'

In [60]:
# 낱개 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,20000):
        if (userdic.missprint[index2] == vocab[index1]):
            vocab[index1] = vocab[index1].replace(userdic.missprint[index2],userdic.standard[index2], True)            
            index2+=1
        else:
            index1+=1       

In [ ]:
# 붙어 있는 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,20000):
        if (userdic.missprint[index2] == vocab[index1].str.contains(usedic.missprint[index2])):
            vocab[index1] = vocab[index1].replace(userdic.misspring[index2],userdic.standard[index2], True)
            index2+=1
        else:
            index1+=1

In [63]:
vocab.str.contains(usedic.missprint[0])

AttributeError: 'list' object has no attribute 'str'

In [64]:
type(vocab)

list

In [184]:
vocab[300:400]

['감형',
 '감형 소음',
 '감형 소음 발생',
 '강변',
 '강변 도로',
 '강변 도로 부근',
 '강시',
 '강시 마찰',
 '강시 마찰 소음',
 '강시 소음',
 '강시 소음 발생',
 '강시 윈도우',
 '강시 윈도우 몰딩',
 '강제',
 '강제 동시',
 '강제 동시 작동',
 '강제 동시 정상',
 '강제 손상',
 '강제 손상 추정',
 '강제 작동',
 '강제 작동 하나',
 '강제 재생',
 '강제 재생 차압',
 '강제 출력',
 '강제 출력 제한',
 '강제 케이블',
 '강제 케이블 풀림',
 '강제 헤드',
 '강제 헤드 램프',
 '개도',
 '개도 목표',
 '개도 목표 이상',
 '개선',
 '개선 고속',
 '개선 고속 주행',
 '개선 교환',
 '개선 불량',
 '개선 불량 교환',
 '개선 사용',
 '개선 사용 교환',
 '개선 요망',
 '개선 요망 사항',
 '개선 요망 임시',
 '개선 운전',
 '개선 운전 밸트라',
 '개선 작업',
 '개선 작업 완료',
 '개선 조속',
 '개선 조속 개선',
 '개선 조치',
 '개선 조치 내용',
 '개선 천장',
 '개선 천장 헤드',
 '개선 필요',
 '개선 확인',
 '개선 휠바',
 '개선 휠바 란스',
 '개조',
 '개조 부분',
 '개조 부분 조치',
 '개조 부품',
 '개조 부품 흔적',
 '개조 분해',
 '개조 분해 흔적',
 '개조 사항',
 '개조 사항 센터',
 '개조 사항 컨트롤',
 '개조 유무',
 '개조 유무 점검',
 '개조 흔적',
 '개조 흔적 점검',
 '개페시',
 '개페시 소음',
 '개페시 소음 발생',
 '개페시 질감',
 '개페시 질감 조치',
 '개폐',
 '개폐 경고',
 '개폐 경고 저음',
 '개폐 고부',
 '개폐 고부 저음',
 '개폐 끄르륵',
 '개폐 끄르륵 끄르륵',
 '개폐 동시',
 '개폐 동시 이음',
 '개폐 레일',
 '개폐 레일 커브',
 '개폐 미터',
 '개폐 미터 표시',
 '개

In [173]:
vocab[170] = vocab[170].replace(userdic.missprint[3],userdic.standard[3], True)
vocab[170]

'간격'

In [175]:
vocab[1] == userdic.missprint[2]

False

In [98]:
dist = np.sum(train_feature_vector, axis=0)
dist.shape

(1, 20000)

In [99]:
dist_pd = pd.DataFrame(dist, columns=vocab)
dist_pd

가감  가감 실내  가감 실내 대쉬보드  가감 저속  가감 저속 등판  가끔  가끔 소음  가끔 소음 허브  가끔 재현  \
0   5     10           3      5         3  15      4         4    199   

   가끔 재현 발생    ...     힌지 볼트  힌지 볼트 볼트  힌지 부위  힌지 부위 수석  힌지 스톱  힌지 스톱 볼트  \
0         4    ...         3         9      5         5      6        38   

   힌지 스트라이크  힌지 스트라이크 조정  힌지 조정  힌지 조정 작업  
0         3            3      5         5  

[1 rows x 20000 columns]

In [100]:
dist_pd.columns

Index(['가감', '가감 실내', '가감 실내 대쉬보드', '가감 저속', '가감 저속 등판', '가끔', '가끔 소음',
       '가끔 소음 허브', '가끔 재현', '가끔 재현 발생',
       ...
       '힌지 볼트', '힌지 볼트 볼트', '힌지 부위', '힌지 부위 수석', '힌지 스톱', '힌지 스톱 볼트',
       '힌지 스트라이크', '힌지 스트라이크 조정', '힌지 조정', '힌지 조정 작업'],
      dtype='object', length=20000)

In [101]:
dist_pd_t = dist_pd.T
dist_pd_t.head()

0
가감           5
가감 실내       10
가감 실내 대쉬보드   3
가감 저속        5
가감 저속 등판     3

In [102]:
dist_pd_t.index.name = "word"

In [103]:
dist_pd_t[:2]

0
word     
가감      5
가감 실내  10

In [104]:
dist_pd_t.reset_index(level='word', inplace=True)

In [105]:
dist_pd_t[:3]

word   0
0          가감   5
1       가감 실내  10
2  가감 실내 대쉬보드   3

In [106]:
dist_pd_t.rename(columns={0:'count'}, inplace = True)

In [107]:
dist_pd_t[:200]

word  count
0            가감      5
1         가감 실내     10
2    가감 실내 대쉬보드      3
3         가감 저속      5
4      가감 저속 등판      3
5            가끔     15
6         가끔 소음      4
7      가끔 소음 허브      4
8         가끔 재현    199
9      가끔 재현 발생      4
10           가나      3
11        가나 왼쪽     39
12     가나 왼쪽 소리      3
13       가나 음으로      6
14    가나 음으로 스티      4
15           가능      3
16        가능 거리      6
17     가능 거리 달라      3
18     가능 거리 변동      3
19        가능 도어      6
20     가능 도어 인너      3
21          가니쉬      3
22       가니쉬 고정      3
23    가니쉬 고정 불량      4
24       가니쉬 고함      3
25   가니쉬 고함 가니쉬      3
26       가니쉬 교체     15
27       가니쉬 교환      4
28    가니쉬 교환 정상      7
29    가니쉬 교환 조치      4
..          ...    ...
170          간극      3
171       간극 범퍼      8
172    간극 범퍼 본넷      5
173       간극 불량     49
174    간극 불량 소음      3
175    간극 불량 점검      4
176    간극 불량 조정      3
177       간극 아래      3
178    간극 아래 범퍼      3
179       간극 조정      8
180    간극 조정 시운      5
181   간극 조정 시운전     23
182    간극 조정 우측      3
183    간극 조정 타이     12
184       간극 조치     47
185    간극 조치 내용      4
186      간극 협소하      3
187   간극 협소하 도어     15
188       간극 확인      3
189    간극 확인 운전      4
190       간극 후드      3
191   간극 후드 앞범퍼      5
192          간단      5
193       간단 조정     14
194    간단 조정 작업      5
195          간섭      3
196       간섭 가속      4
197    간섭 가속 소음     32
198       간섭 도어     11
199    간섭 도어 도장      8

[200 rows x 2 columns]

In [260]:
dist_pd_t = dist_pd_t.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="개스",value="가스")
df1 = df.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="간단차",value="간격 단차")
df1 = df.replace(to_replace="갭단차",value="간격 단차")
df1 = df.replace(to_replace="단간차",value="간격 단차")
df1 = df.replace(to_replace="간혈적",value="간헐적")
df1 = df.replace(to_replace="감지센서",value="감지 센서")
df1 = df.replace(to_replace="울트라센서",value="감지 센서")
df1 = df.replace(to_replace="울트라 센서",value="감지 센서")
df1 = df.replace(to_replace="개페",value="개폐")
df1 = df.replace(to_replace="개패",value="개폐")
df1 = df.replace(to_replace="문구",value="고장코드")
df1 = df.replace(to_replace="맨트",value="고장코드")
df1 = df.replace(to_replace="멘트",value="고장코드")

In [470]:
aa = vectorizer.fit_transform(dist_pd_t['word'])

In [472]:
aa.shape

(20000, 20000)

In [261]:
dist_pd_t[:200]

word  count
0            가감      5
1         가감 실내     10
2    가감 실내 대쉬보드      3
3         가감 저속      5
4      가감 저속 등판      3
5            가끔     15
6         가끔 소음      4
7      가끔 소음 허브      4
8         가끔 재현    199
9      가끔 재현 발생      4
10           가나      3
11        가나 왼쪽     39
12     가나 왼쪽 소리      3
13       가나 음으로      6
14    가나 음으로 스티      4
15           가능      3
16        가능 거리      6
17     가능 거리 달라      3
18     가능 거리 변동      3
19        가능 도어      6
20     가능 도어 인너      3
21          가니쉬      3
22       가니쉬 고정      3
23    가니쉬 고정 불량      4
24       가니쉬 고함      3
25   가니쉬 고함 가니쉬      3
26       가니쉬 교체     15
27       가니쉬 교환      4
28    가니쉬 교환 정상      7
29    가니쉬 교환 조치      4
..          ...    ...
170          간격      3
171       간극 범퍼      8
172    간극 범퍼 본넷      5
173       간극 불량     49
174    간극 불량 소음      3
175    간극 불량 점검      4
176    간극 불량 조정      3
177       간극 아래      3
178    간극 아래 범퍼      3
179       간극 조정      8
180    간극 조정 시운      5
181   간극 조정 시운전     23
182    간극 조정 우측      3
183    간극 조정 타이     12
184       간극 조치     47
185    간극 조치 내용      4
186      간극 협소하      3
187   간극 협소하 도어     15
188       간극 확인      3
189    간극 확인 운전      4
190       간극 후드      3
191   간극 후드 앞범퍼      5
192          간단      5
193       간단 조정     14
194    간단 조정 작업      5
195          간섭      3
196       간섭 가속      4
197    간섭 가속 소음     32
198       간섭 도어     11
199    간섭 도어 도장      8

[200 rows x 2 columns]

# tf-idf 가중치 적용

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [49]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(7676, 20000)

In [50]:
#%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

(853, 20000)

# 머신러닝 : 학습

In [51]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [52]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 1.63 s


# 모델평가

In [53]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 27.3 s


array([0.80078125, 0.80781759, 0.78501629, 0.80065147, 0.80325733])

In [54]:
round(np.mean(score)*100,2)

79.95

# 예측

In [55]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'X', 'X'], dtype=object)

In [56]:
y_pred.shape

(853,)

In [57]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0             X
1             X
2             X
3             X
4             X
5             X
6             X
7             X
8             X
9             X

In [58]:
df_test['category_pred'] = y_pred

# 결과검증

In [59]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
124                W  2015     20151103         ZZ7         B11   
1073               W  2017     20170828         ZZ7         B14   
3054               W  2016     20160906         ZZ5         I12   
1527               W  2018     20180831         ZZ7         B17   
867                W  2018     20180905         ZZ7         B13   

                ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
124           조립 문제_미장착/오장착         KV11          YP     A9          TT   
1073             조립 문제_간/단차         KV11          YP     A9          CC   
3054  작동불량_작동이 원활하지 않음(무거움)         KV11          YP     A9          TT   
1527           조립 문제_정위치 불량         KV11          CK     J5          AC   
867             조립 문제_유격/유동         KV11          YP     A9          CC   

     repn_part_cd                                       clm_prtl_sbc 대상 선정  \
124         85765       러기    네트    훅    미장    착    스    크류    적도...     X   
1073        12441       리    어도    발생    조치    리    휀더    판넬    스...     X   
3054        83362    리    어도    커튼    문제    점검    결과    커튼    오른쪽...     X   
1527        95811       후    측방    고등    표    점검    내용    및    조치...     X   
867         56115       주행    휠    보스    유격    발생    점검    및     ...     O   

          구분   책임구    원인                검토결과 target_pred category_pred  \
124        N     N     N                X---           X             X   
1073       N     N     N                X---           X             X   
3054       N     N     N                X---           X             X   
1527       N     N     N                X---           X             X   
867   실내부품이음  생산공장  조립불량  O-실내부품이음-생산공장-조립불량           O             X   

      pred_diff  
124           1  
1073          1  
3054          1  
1527          1  
867           0

In [60]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

577

In [61]:
y_pred_len = len(y_pred)
y_pred_len

853

# 정확도

In [62]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

67.6